In [50]:
import pandas as pd
import warnings
warnings.simplefilter('ignore')
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')

In [51]:
year_months = list(uselog_months['연월'].unique())
uselog = pd.DataFrame()
for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months['연월']==year_months[i]]
    tmp.rename(columns={'count':'count_0'}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months['연월']==year_months[i-1]]
    del tmp_before['연월'] #del 연월 for second df because column exists in first tmp
    tmp_before.rename(columns={'count': 'count_1'}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on='customer_id', how='left')
    uselog = pd.concat([uselog, tmp], ignore_index=True)
uselog.head()

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


In [31]:
uselog.loc[uselog['customer_id']=='AS002855']

,연월,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
3063,201806,AS002855,5,5.0
6167,201807,AS002855,5,5.0
9300,201808,AS002855,3,5.0
12433,201809,AS002855,7,3.0
15541,201810,AS002855,3,7.0
18611,201811,AS002855,5,3.0
21656,201812,AS002855,2,5.0
24769,201901,AS002855,6,2.0
27861,201902,AS002855,5,6.0


In [52]:
from dateutil.relativedelta import relativedelta
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)
exit_customer["연월"] = pd.to_datetime(exit_customer["exit_date"]).dt.strftime("%Y%m")
#exit_customer['연월'] = exit_customer['exit_date'].dt.strftime('%Y%m') wrong
uselog["연월"] = uselog["연월"].astype(str)
exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "연월"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
exit_uselog = exit_uselog.dropna(subset=['name'])
print(len(exit_uselog))
print(len(exit_uselog['customer_id'].unique()))
exit_uselog.head()

1104
1104


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30 00:00:00
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,일반,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30 00:00:00
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30 00:00:00
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,일반,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30 00:00:00
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,일반,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30 00:00:00


In [61]:
conti_customer = customer.loc[customer['is_deleted']==0]
conti_uselog = pd.merge(uselog, conti_customer, on=['customer_id'], how='left')
print(len(conti_uselog))
conti_uselog = conti_uselog.dropna(subset=['name'])
print(len(conti_uselog))

33851
27422


In [63]:
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
conti_uselog = conti_uselog.drop_duplicates(subset='customer_id')
print(len(conti_uselog))
conti_uselog.head()

2842


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201808,OA250792,5,6.0,XXXX,C01,M,2017-06-01,NaN,CA1,...,종일,10500.0,일반,5.750000,6.0,7.0,3.0,1.0,2019-04-30,22.0
1,201903,TS599859,9,11.0,XXXXXX,C03,M,2019-01-09,NaN,CA1,...,야간,6000.0,일반,11.000000,11.0,13.0,9.0,1.0,2019-04-30,3.0
2,201808,IK352765,8,9.0,XXXX,C02,F,2018-03-01,NaN,CA1,...,주간,7500.0,일반,7.333333,7.0,9.0,6.0,1.0,2019-04-30,13.0
3,201811,OA607825,6,6.0,XXXXX,C03,M,2016-12-01,NaN,CA1,...,야간,6000.0,일반,5.333333,5.0,7.0,4.0,1.0,2019-04-30,28.0
4,201806,OA266742,3,5.0,XXXXX,C02,M,2015-07-01,NaN,CA1,...,주간,7500.0,일반,5.583333,5.5,8.0,3.0,1.0,2019-04-30,45.0


In [64]:
predict_data = pd.concat([conti_uselog, exit_uselog], ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201808,OA250792,5,6.0,XXXX,C01,M,2017-06-01,NaN,CA1,...,10500.0,일반,5.750000,6.0,7.0,3.0,1.0,2019-04-30,22.0,NaN
1,201903,TS599859,9,11.0,XXXXXX,C03,M,2019-01-09,NaN,CA1,...,6000.0,일반,11.000000,11.0,13.0,9.0,1.0,2019-04-30,3.0,NaN
2,201808,IK352765,8,9.0,XXXX,C02,F,2018-03-01,NaN,CA1,...,7500.0,일반,7.333333,7.0,9.0,6.0,1.0,2019-04-30,13.0,NaN
3,201811,OA607825,6,6.0,XXXXX,C03,M,2016-12-01,NaN,CA1,...,6000.0,일반,5.333333,5.0,7.0,4.0,1.0,2019-04-30,28.0,NaN
4,201806,OA266742,3,5.0,XXXXX,C02,M,2015-07-01,NaN,CA1,...,7500.0,일반,5.583333,5.5,8.0,3.0,1.0,2019-04-30,45.0,NaN


In [67]:
predict_data['period'] = 0
predict_data['now_date'] = pd.to_datetime(predict_data['연월'], format='%Y%m')
predict_data['start_date']= pd.to_datetime(predict_data['start_date'])
for i in range(len(predict_data)):
    delta = relativedelta(predict_data['now_date'][i], predict_data['start_date'][i])
    predict_data['period'][i] = int(delta.years*12 + delta.months)
predict_data.head()

,연월,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201808,OA250792,5,6.0,XXXX,C01,M,2017-06-01,NaN,CA1,...,5.750000,6.0,7.0,3.0,1.0,2019-04-30,22.0,NaN,14,2018-08-01
1,201903,TS599859,9,11.0,XXXXXX,C03,M,2019-01-09,NaN,CA1,...,11.000000,11.0,13.0,9.0,1.0,2019-04-30,3.0,NaN,1,2019-03-01
2,201808,IK352765,8,9.0,XXXX,C02,F,2018-03-01,NaN,CA1,...,7.333333,7.0,9.0,6.0,1.0,2019-04-30,13.0,NaN,5,2018-08-01
3,201811,OA607825,6,6.0,XXXXX,C03,M,2016-12-01,NaN,CA1,...,5.333333,5.0,7.0,4.0,1.0,2019-04-30,28.0,NaN,23,2018-11-01
4,201806,OA266742,3,5.0,XXXXX,C02,M,2015-07-01,NaN,CA1,...,5.583333,5.5,8.0,3.0,1.0,2019-04-30,45.0,NaN,35,2018-06-01


In [68]:
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1               258
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [69]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

연월                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2636
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2636
period                  0
now_date                0
dtype: int64

In [70]:
target_col = ['campaign_name', 'class_name', 'gender', 'count_1', 'routine_flg', 'period', 'is_deleted']
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,일반,종일,M,6.0,1.0,14,0.0
1,일반,야간,M,11.0,1.0,1,0.0
2,일반,주간,F,9.0,1.0,5,0.0
3,일반,야간,M,6.0,1.0,23,0.0
4,일반,주간,M,5.0,1.0,35,0.0


In [71]:
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_일반,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_야간,class_name_종일,class_name_주간,gender_F,gender_M
0,6.0,1.0,14,0.0,1,0,0,0,1,0,0,1
1,11.0,1.0,1,0.0,1,0,0,1,0,0,0,1
2,9.0,1.0,5,0.0,1,0,0,0,0,1,1,0
3,6.0,1.0,23,0.0,1,0,0,1,0,0,0,1
4,5.0,1.0,35,0.0,1,0,0,0,0,1,0,1


In [72]:
del predict_data['campaign_name_일반']
del predict_data['class_name_야간']
del predict_data['gender_M']
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_입회비무료,campaign_name_입회비반액할인,class_name_종일,class_name_주간,gender_F
0,6.0,1.0,14,0.0,0,0,1,0,0
1,11.0,1.0,1,0.0,0,0,0,0,0
2,9.0,1.0,5,0.0,0,0,0,1,1
3,6.0,1.0,23,0.0,0,0,0,0,0
4,5.0,1.0,35,0.0,0,0,0,1,0


In [73]:
from sklearn.tree import DecisionTreeClassifier
import sklearn.model_selection

exit = predict_data.loc[predict_data['is_deleted']==1]
conti = predict_data.loc[predict_data['is_deleted']==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 0.
 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 0.
 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1.
 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1.
 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 0. 1.
 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 1.

In [77]:
results_test = pd.DataFrame({'y_test':y_test, 'y_pred':y_test_pred})
results_test.head()

,y_test,y_pred
120,1.0,1.0
1606,0.0,0.0
507,1.0,1.0
462,1.0,1.0
436,1.0,1.0


In [78]:
correct = len(results_test.loc[results_test['y_test']==results_test['y_pred']])
data_count = len(results_test)
score_test = correct / data_count
print(score_test)

0.9106463878326996


In [79]:
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9106463878326996
0.9803548795944234


In [80]:
#트리의 깊이를 얕게 하면 모델을 단순화할 수 있다
X = pd.concat([exit,conti], ignore_index=True)
y = X['is_deleted']
del X['is_deleted']
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9125475285171103
0.9233206590621039


In [81]:
#model.feature_importances_ show percentage distrib. of importances of each variable
importance = pd.DataFrame({'feature_names':X.columns, 'coefficient': model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.346968
1,routine_flg,0.123305
2,period,0.529571
3,campaign_name_입회비무료,0.000000
4,campaign_name_입회비반액할인,0.000000
5,class_name_종일,0.000000
6,class_name_주간,0.000156
7,gender_F,0.000000


In [82]:
count_1 = 3
routine_flg = 1
period = 10
campaign_name = '입회비무료'
class_name = '종일'
gender = 'M'

In [83]:
if campaign_name == "입회비반값할인":
    campaign_name_list = [1, 0]
elif campaign_name == "입회비무료":
    campaign_name_list = [0, 1]
elif campaign_name == "일반":
    campaign_name_list = [0, 0]
if class_name == "종일":
    class_name_list = [1, 0]
elif class_name == "주간":
    class_name_list = [0, 1]
elif class_name == "야간":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)
# list.extend(list2) concatenates list

In [84]:
input_data

[3, 1, 10, 0, 1, 1, 0, 0]